In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from dotenv import load_dotenv
load_dotenv()
from sql_components import setup_database
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.language_models import BaseChatModel
import json

In [ ]:
class GraphState(TypedDict):
    question: str
    language: str
    sql_query: str
    query_result: str
    final_answer: str
    #messages: Annotated[list[BaseMessage], object]

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# Create the graph
workflow = StateGraph(GraphState)
print("Workflow is created:", workflow)

Workflow is created: <langgraph.graph.state.StateGraph object at 0x00000192C2CB17C0>


In [ ]:
# Translation prompt
def create_translation_to_english_prompt():
    return ChatPromptTemplate.from_messages([
        ("system", "You are a professional translator. Translate the given text from {input_language} to English, preserving the original meaning and context."),
        ("human", "Text to translate: {input_text}")
    ])

In [ ]:
def detect_language(state: GraphState):
    input_language = state.get('language', 'English')
    
    if input_language.lower() != 'english':
        translation_prompt = create_translation_to_english_prompt()
        translator = translation_prompt | llm
        
        translation_result = translator.invoke({
            "input_language": input_language,
            "input_text": state['question']
        })
        
        return {
            
            "question": translation_result.content
            #"messages": state.get('messages', []) + 
                   #     [HumanMessage(content=f"Original query in {input_language}: {state['question']}")]
        }
    
    return {"question": state['question']}

In [ ]:
# Add nodes
workflow.add_node("detect_language",detect_language)
# Define edges
workflow.add_edge(START, "detect_language")
workflow.add_edge("detect_language", END)
print("Graph is created")

Graph is created


In [ ]:
# Modified detect_language function
graph_builder = workflow.compile()


In [ ]:
result=graph_builder.invoke({"question":"Bonjour ! Comment ça va ?",
                      "language":"French",
                      "sql_query":"",
                      "query_result":"",
                      "final_answer":""})

In [ ]:
result['question']

'Hello! How are you?'

In [ ]:
import re

In [ ]:
def extract_sql_command(content: str) -> str:
    """
    Extract SQL command from the given content.
    
    Args:
        content (str): The input text containing the SQL command
    
    Returns:
        str: The extracted SQL command
    """
    # Remove code block markers if present
    # Use regex to extract SQL command within ```sql ``` or just extract the SQL
    match = re.search(r'```sql\n(.*?)```', content, re.DOTALL)
    
    if match:
        # If found within code block
        return match.group(1).strip()
    
    # If no code block, try to extract SQL directly
    match = re.search(r'SELECT.*', content, re.DOTALL)
    
    if match:
        return match.group(0).strip()
    
    # If no SQL found
    return ""



In [ ]:
content='```sql\nSELECT COUNT(*) AS totalProducts\nFROM products\nLIMIT 10;\n```'

sql_command = extract_sql_command(content)
print("Extracted SQL command:", sql_command)

Extracted SQL command: SELECT COUNT(*) AS totalProducts
FROM products
LIMIT 10;


In [ ]:
result = "There are 10 products in total."
question = "How many Products are there?"

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
import json

In [ ]:
chart_prompt = ChatPromptTemplate.from_messages([
    ("system","""You are an expert data visualization consultant. 
Given a database query result, you must:
1. Recommend the most appropriate chart type
2. Suggest a meaningful DataFrame structure to visualize the data

Chart type recommendations:
- Bar Chart: Good for comparing categories, showing counts
- Pie Chart: Ideal for showing parts of a whole
- Donut Chart: Similar to pie chart, but with a center hole
- Line Chart: Best for showing trends over time

DataFrame structure should include meaningful columns that help 
visualize the data effectively.

Database Query Result: {result}
Original Question: {question}

Please provide:
1. Recommended chart type
2. DataFrame columns and values to best represent this data

Response format:
```json
{{
    "chart_type": "bar|pie|donut|line",
    "dataframe_structure": {{
        "columns": ["col1", "col2"],
        "values": [...]
    }}
}}
```""")
])
response = llm.generate(chart_prompt.format(result=result, question=question))
print("Response:", response)
response_data = json.loads(response)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: Monthly unique traces usage limit exceeded"}\n')


KeyboardInterrupt: 

In [ ]:
chart_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert data visualization consultant. Given a database query result, recommend the most appropriate chart type and suggest a meaningful DataFrame structure to visualize the data. Database Query Result: {result} Original Question: {question} Response format: {chart_type: 'bar|pie|donut|line', dataframe_structure: {columns: ['col1', 'col2'], values: [...]}}")
])

def generate_response(result, question):
    response = llm.generate(chart_prompt.format(result=result, question=question))
    print("Response:", response)
    response_data = json.loads(response)
    return response_data

# Example usage
result = "There are 10 products in total."
question = "How many Products are there?"
response_data = generate_response(result, question)

KeyError: 'chart_type'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
chart_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert data visualization consultant. Given a database query result, recommend the most appropriate chart type and suggest a meaningful DataFrame structure to visualize the data. Database Query Result: {result} Original Question: {question} Response format: {chart_type: 'bar|pie|donut|line', dataframe_structure: {columns: ['col1', 'col2'], values: [...]}}")
])

In [ ]:
# Example usage
result = "There are 10 products in total."
question = "How many Products are there?"

In [ ]:
response = llm.invoke(chart_prompt.format(result=result, question=question))

KeyError: 'chart_type'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import json

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    
    # Create a prompt template with JSON output parser
chart_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert data visualization consultant. 
        Analyze the following information and provide a detailed recommendation:
        
        Database Query Result: {result}
        Original Question: {question}
        
        Your task is to:
        1. Recommend the most appropriate chart type
        2. Suggest a meaningful DataFrame structure
        
        Respond in a strict JSON format with these keys:
        - chart_type: Choose from 'bar', 'pie', 'donut', or 'line'
        - dataframe: Include columns and data
        
        Example Response:
        {
            "chart_type": "bar",
            "dataframe": {
                "columns": ["Category", "Value"],
                "data": [
                    ["Products", 10]
                ]
            }
        }""")
    ])

In [ ]:
 # Create an output parser
json_parser = JsonOutputParser()
    
    # Create a chain
chain = chart_prompt | llm | json_parser

In [ ]:
visualization_details = chain.invoke({
            "result": result,
            "question": question
        })

KeyError: 'Input to ChatPromptTemplate is missing variables {\'\\n            "chart_type"\'}.  Expected: [\'\\n            "chart_type"\', \'question\', \'result\'] Received: [\'result\', \'question\']\nNote: if you intended {\n            "chart_type"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n            "chart_type"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import json

def generate_visualization_details(result, question):
    # Initialize the LLM
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    
    # Create a prompt template
    chart_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert data visualization consultant. 
        Analyze the following information and provide a detailed recommendation:
        
        Database Query Result: {result}
        Original Question: {question}
        
        Your task is to:
        1. Recommend the most appropriate chart type
        2. Suggest a meaningful DataFrame structure
        
        Respond ONLY with a JSON object containing:
        - A chart type (bar/pie/donut/line)
        - A DataFrame structure
        
        Example:
        {
            "chart_type": "bar",
            "dataframe": {
                "columns": ["Category", "Value"],
                "data": [["Products", 10]]
            }
        }""")
    ])
    
    # Generate response
    try:
        # Create a chain
        chain = chart_prompt | llm
        
        # Invoke the chain
        response = chain.invoke({
            "result": result,
            "question": question
        })
        
        # Parse the JSON response
        try:
            # Try to parse the content as JSON
            visualization_details = json.loads(response.content)
            return visualization_details
        except json.JSONDecodeError:
            # If JSON parsing fails, print raw response
            print("Failed to parse JSON. Raw response:")
            print(response.content)
            return None
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Comprehensive alternative approach
def generate_visualization_details_v2(result, question):
    # Initialize the LLM
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    
    # Detailed prompt as a string
    prompt_text = f"""You are an expert data visualization consultant.

Database Query Result: {result}
Original Question: {question}

Provide a JSON response with:
1. Recommended chart type (bar/pie/donut/line)
2. DataFrame structure for visualization

Respond EXACTLY in this format:
{{
    "chart_type": "bar",
    "dataframe": {{
        "columns": ["Category", "Value"],
        "data": [["Products", 10]]
    }}
}}"""
    
    # Generate response
    try:
        # Directly invoke the LLM
        response = llm.invoke(prompt_text)
        
        # Parse the JSON response
        try:
            visualization_details = json.loads(response.content)
            return visualization_details
        except json.JSONDecodeError:
            print("Failed to parse JSON. Raw response:")
            print(response.content)
            return None
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
result = "There are 10 products in total."
question = "How many Products are there?"

# Try both methods
print("Method 1:")
visualization_details = generate_visualization_details(result, question)

print("\nMethod 2:")
visualization_details_v2 = generate_visualization_details_v2(result, question)

# Print results
def print_visualization_details(details):
    if details:
        print("Chart Type:", details.get('chart_type'))
        print("DataFrame Structure:", details.get('dataframe'))
    else:
        print("No visualization details generated.")

print("\nMethod 1 Results:")
print_visualization_details(visualization_details)

print("\nMethod 2 Results:")
print_visualization_details(visualization_details_v2)


Method 1:
An error occurred: 'Input to ChatPromptTemplate is missing variables {\'\\n            "chart_type"\'}.  Expected: [\'\\n            "chart_type"\', \'question\', \'result\'] Received: [\'result\', \'question\']\nNote: if you intended {\n            "chart_type"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n            "chart_type"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

Method 2:

Method 1 Results:
No visualization details generated.

Method 2 Results:
Chart Type: bar
DataFrame Structure: {'columns': ['Category', 'Value'], 'data': [['Products', 10]]}


In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

result = "There are 10 products in total."
question = "How many Products are there?"   

# Detailed prompt as a string
prompt_text = f"""You are an expert data visualization consultant.

Database Query Result: {result}
Original Question: {question}

Provide a JSON response with:
1. Recommended chart type (bar/pie/donut/line)
2. DataFrame structure for visualization

Respond EXACTLY in this format:
{{
    "chart_type": "bar",
    "dataframe": {{
        "columns": ["Category", "Value"],
        "data": [["Products", 10]]
    }}
}}"""

response = llm.invoke(prompt_text)
visualization_details = json.loads(response.content)

if visualization_details:
    print("Chart Type:", visualization_details.get('chart_type'))
    print("DataFrame Structure:", visualization_details.get('dataframe'))
else:
    print("No visualization details generated.")